# Telecom Customer Churn Prediction

Compaines are always trying to save money. On that note did you know that attracting a new customer costs approximately 5 times as much as keeping an existing one.



# Overview

1. [Got to Loading libraries and data](#1-loading-libraries-and-data)



2. [Got to Exploratory Data Analysis](#2-exploring-the-data)



3. [Go to Feature Engineering](#3-feature-engineering)

[Go to Stopped Here!](#stopped-here)



# 1. Loading libraries and data

In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [3]:
# Load Data
df = pd.read_csv('/Users/vcarrillo/Springboard/Springboard-1/Telco-Customer-Churn.csv')

# 2. Exploring the Data

Each row represents a customer, each column contains customer's attributes described on the column.

In [4]:
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


Data set features:

- Customer who left within the last month - column is named 'Churn'
- Services that each customer has signed up for - phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies.
- Customer account information - how long they have been a customer in years ('tenure'), payment method, type of contract they have (month-to-month, year, ect), monthly charges and total charges.
- Demographic information about customers - gender, age range, and if they have partners and dependents. 

In [5]:
df.shape

(7043, 21)

Dataset shape:
- This data set has 7043 rows and 21 columns. 

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
# Count how many of each data type there are in df.
df.dtypes.value_counts()

object     18
int64       2
float64     1
Name: count, dtype: int64

Data type counts:
- This data set contains 18 columns that are object type. I will have to manipulate these later to prepare for machine learning.
- Two columns are int and 1 is float. I might have to scale these during preprocessing for machine learning.
- TotalCharges is an object dtype when it should be numeric. I will have to deal with this during feature engineering. 

In [8]:
df.columns.values

array(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'Churn'], dtype=object)

- Target value during EDA will be Churn.

In [9]:
# Check for missing data in data set
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

There are no missing values in this data set. 

In [10]:
# Drop customerID since we will not need this data for this project. 
df = df.drop(['customerID'], axis = 1)

In [11]:
df.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [12]:
# Changing TotalCharges to numeric

df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')

In [13]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [14]:
df.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

- Now that TotalCharges is a numeric data type we can see that it has 11 missing values. I have to deal with these before applying machine learning algorythms. 

In [15]:
df[np.isnan(df['TotalCharges'])]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


- Upon further investigation I can see that these customers have been customers for less than a month which implies that they are new customers.
- Since they are new customers that means that the monthly charge should also be the TotalCharge.
- However since it is only 11 customers and this represents a minute amount of the customers in data set. I can drop these. 

In [16]:
# dropping missing values
df = df.dropna()

In [17]:
df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

# EDA
- Churn and tenure? (Hypothesis Testing)
- Churn and Gender? (Hypothesis Testing)
- Churn and SeniorCitizen status? (Hypothesis Testing)
- Churn and Dependents? (Hypothesis Testing)
- Churn and payment method? (Hypothesis Testing)
- Churn and Internet Service? (Hypothesis Testing)

## Churn Contract Type and Average Monthly Charges

In [53]:
Churn_Contract_Charges = df.groupby(['Churn', 'Contract'])['MonthlyCharges'].mean().reset_index()

Churn_Contract_Charges.head(6)

,Churn,Contract,MonthlyCharges
0,No,Month-to-month,61.462635
1,No,One year,62.540926
2,No,Two year,60.112798
3,Yes,Month-to-month,73.019396
4,Yes,One year,85.050904
5,Yes,Two year,86.777083


In [54]:

fig = px.bar(
    Churn_Contract_Charges,
    x='Contract',
    y='MonthlyCharges',
    color='Churn',
    barmode='group',
    title='Average Monthly Charges by Contract Type and Churn Status',
    labels={'MonthlyCharges': 'Avg Monthly Charges ($)'}
)
fig.update_traces(
    texttemplate='%{y:.2f}',
    textposition='outside'
)
fig.update_layout(
    yaxis=dict(showgrid=True, gridcolor='gray'),
    font=dict(size=16),
    width=800,
    height=600
)

fig.show()


In [55]:
Churn_Contract_Charges.head(6)

,Churn,Contract,MonthlyCharges
0,No,Month-to-month,61.462635
1,No,One year,62.540926
2,No,Two year,60.112798
3,Yes,Month-to-month,73.019396
4,Yes,One year,85.050904
5,Yes,Two year,86.777083


## Observations: Churn, Contract Type and Average Monthly Charges
- On average customers who churned were paying more per month than those who did not, this was true regardless of the type of contract.
- Customers who were month-to-month and churned were paying on average $12 more per month than those who did not churn.
- Customers who were on a year conctract and churned were paying on average $23 more than customers on a year contract who did not churn. 
- Customers who churned and on a two year contract were on average paying $26 more than customers who did not churn and were on a two year contract. 

## Churn and Contract Type

In [56]:
Churn_Contract_Charges = df.groupby(['Churn', 'Contract']).size().reset_index(name='count')

In [58]:
Churn_Contract_Charges.head()

,Churn,Contract,count
0,No,Month-to-month,2220
1,No,One year,1306
2,No,Two year,1637
3,Yes,Month-to-month,1655
4,Yes,One year,166


In [57]:

fig = px.sunburst(
    Churn_Contract_Charges, 
    path=['Churn', 'Contract'], 
    values='count', 
    title='Customer Distribution by Churn and Contract Type'
)

# Update to show percentages along with labels
fig.update_traces(textinfo='label+value +percent entry')
font=dict(size=40),
fig.update_layout(width=800, height=800)

fig.show()

- Churn and tenure? (Hypothesis Testing)
- Churn and Gender? (Hypothesis Testing)
- Churn and SeniorCitizen status? (Hypothesis Testing)
- Churn and Dependents? (Hypothesis Testing)
- Churn and payment method? (Hypothesis Testing)
- Churn and Internet Service? (Hypothesis Testing)

# Observation: Churn and Contract Type
- 27% of Telco customers churned. 
- Of the 1869 customers (27%) who churned, 1655 (24%) of these were on a Month-to-month contract. 
- This is a cleary indicates that customers at risk of churning are on a Month-to-month contract. 
- Of the total customers who churned only 2% were on a year contract.
- Of the customers who churned only 1% were on a Two year contract.  

## Churn and Tenure

In [64]:
Churn_Tenure = df.groupby(['Churn', 'tenure']).size().reset_index(name='count')

In [73]:
Churn_Tenure.value_counts()

Churn  tenure  count
No     1       233      1
       2       115      1
Yes    21      17       1
       22      27       1
       23      13       1
                       ..
No     49      51       1
       50      58       1
       51      60       1
       52      72       1
Yes    72      6        1
Name: count, Length: 144, dtype: int64

In [66]:
Churn_Tenure_Charges = df.groupby(['Churn', 'tenure'])['MonthlyCharges'].mean().reset_index()

In [67]:

fig = px.bar(
    Churn_Tenure_Charges,
    x='tenure',
    y='MonthlyCharges',
    color='Churn',
    barmode='group',
    title='Average Monthly Charges by Tenure and Churn Status',
    labels={'MonthlyCharges': 'Avg Monthly Charges ($)'}
)
fig.update_traces(
    texttemplate='%{y:.2f}',
    textposition='outside'
)
fig.update_layout(
    yaxis=dict(showgrid=True, gridcolor='gray'),
    font=dict(size=16),
    width=800,
    height=600
)

fig.show()

## Churn and Payment Method

In [59]:
Churn_Payment_Method = df.groupby(['Churn', 'PaymentMethod']).size().reset_index(name='count')

In [62]:

fig = px.sunburst(
    Churn_Payment_Method, 
    path=['Churn', 'PaymentMethod'], 
    values='count', 
    title='Customer Distribution by Churn and Payemtn Method'
)

# Update to show percentages along with labels
fig.update_traces(textinfo='label +percent entry')
font=dict(size=40),
fig.update_layout(width=800, height=800)

fig.show()

# Observation: Customer Churn and Payment Method
- Of the 15% of customers who churned used Electronic check as their payment method. This far exceeds the other categories of customers who churned which were almost identical. 

In [49]:
fig = px.histogram(df, x="Churn", color="PaymentMethod", title="Customer Payment Method distribution and Churn")
fig.update_layout(width=900, height=600, bargap=0.1)
fig.show()

## Churn and Gender

In [75]:
Churn_Gender = df.groupby(['Churn', 'gender']).size().reset_index(name='count')

In [76]:
fig = px.sunburst(
    Churn_Payment_Method, 
    path=['Churn', 'gender'], 
    values='count', 
    title='Customer Distribution by Churn and Gender'
)

# Update to show percentages along with labels
fig.update_traces(textinfo='label +percent entry')
font=dict(size=40),
fig.update_layout(width=800, height=800)

fig.show()

# Observations: Churn and Gender
- Gender appears to have no association to churn rate. Both categories are roughly identical.

## Churn and Senior citizen status

In [77]:
Churn_SeniorCitizen = df.groupby(['Churn', 'SeniorCitizen']).size().reset_index(name='count')

In [80]:
Churn_SeniorCitizen.head(6)

,Churn,SeniorCitizen,count
0,No,0,4497
1,No,1,666
2,Yes,0,1393
3,Yes,1,476


In [81]:
fig = px.sunburst(
    Churn_SeniorCitizen, 
    path=['Churn', 'SeniorCitizen'], 
    values='count', 
    title='Customer Distribution by Churn and Senior Citizen Status'
)

# Update to show percentages along with labels
fig.update_traces(textinfo='label +value+percent entry')
font=dict(size=40),
fig.update_layout(width=800, height=800)

fig.show()

# Observation:
- Of the 1869 customers that churned, 476 (7%) were Senior Citizens. 

# EDA

- Churn and Dependents? (Hypothesis Testing)
- Churn and Internet Service? (Hypothesis Testing)

## Churn and if customer had dependents

In [83]:
fig= px.histogram(df, x="Churn", color='Dependents', barmode="group", title='Distribution of Churn and Dependents')
fig.update_layout(width=800, height= 500, bargap=0.2)
fig.show()

Hypothesis testing: There is no difference in Churn rate and whether customer has dependents. 

- I will have to find proportion of customers who did not churn and had dependents, as well as the proportion of customers who churned and did/didn't have dependents.

# STOPPED HERE! 

## 06/09

## Notes: 
- I have to fix tenure section and pick a better visualization.
- I have to add hypothesis testing to some sections.
- Do churn and internet service. 

- Next I will perform feature Engineering on binary columns

In [43]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [44]:
df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


# 3 feature engineering 

## Binary columns:
Partner
Dependents
PhoneService
OnlineSecurity
OnlineBackup
DeviceProtection
TechSupport
StreamingTV
StreamingMovies
PaperlessBilling
Churn